<a href="https://colab.research.google.com/github/jihwanK/practice/blob/master/kaggle_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install htop -q
!pip install glances[gpu] catboost optuna dask[dataframe] -q

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libnl-genl-3-200
Suggested packages:
  lm-sensors strace
The following NEW packages will be installed:
  htop libnl-genl-3-200
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 140 kB of archives.
After this operation, 404 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libnl-genl-3-200 amd64 3.5.0-0.1 [12.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 htop amd64 3.0.5-7build2 [128 kB]
Fetched 140 kB in 1s (104 kB/s)
Selecting previously unselected package libnl-genl-3-200:amd64.
(Reading database ... 123588 files and directories currently installed.)
Preparing to unpack .../libnl-genl-3-200_3.5.0-0.1_amd64.deb ...
Unpacking libnl-genl-3-200:amd64 (3.5.0-0.1) ...
Selecting previously unselected package htop.
Preparing to unpack .../htop_3.0.5-7build2_amd64.deb .

In [2]:
from google.colab import userdata
import os
os.environ["KAGGLE_USERNAME"] = userdata.get('kaggle_username')
os.environ["KAGGLE_KEY"] = userdata.get('kaggle_key')
import kaggle

# Preprocess

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, GridSearchCV
from imblearn.over_sampling import SMOTE

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# import cupy as cp
import xgboost as xgb

In [4]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab/practice/dataset/kaggle/s4e7_train.csv", usecols=lambda x: x != 'id')
X_test = pd.read_csv("/content/drive/MyDrive/Colab/practice/dataset/kaggle/s4e7_test.csv", usecols=lambda x: x != 'id')
test_id = pd.read_csv("/content/drive/MyDrive/Colab/practice/dataset/kaggle/s4e7_test.csv", usecols=lambda x: x == 'id')
X, y = train_df.drop('Response', axis=1), train_df.loc[:,'Response']

In [5]:
X["Vehicle_Damage"] = X["Vehicle_Damage"].apply(lambda x: int(x == "Yes"))
X["Gender"] = X["Gender"].apply(lambda x: int(x == "Male"))
X["Vehicle_Age"] = X["Vehicle_Age"].map({"< 1 Year": 0, "1-2 Year": 1, "> 2 Years": 2})

X_test["Vehicle_Damage"] = X_test["Vehicle_Damage"].apply(lambda x: int(x == "Yes"))
X_test["Gender"] = X_test["Gender"].apply(lambda x: int(x == "Male"))
X_test["Vehicle_Age"] = X_test["Vehicle_Age"].map({"< 1 Year": 0, "1-2 Year": 1, "> 2 Years": 2})

In [6]:
numerical_features = ['Age', 'Annual_Premium', 'Vintage']
categorical_features = ['Gender', 'Vehicle_Age', 'Vehicle_Damage', 'Region_Code', 'Policy_Sales_Channel']

In [7]:
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])

X[categorical_features] = X[categorical_features].astype(np.int64).astype("category")
X_test[categorical_features] = X_test[categorical_features].astype(np.int64).astype("category")

X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, shuffle=True, test_size=30000, random_state=42)

In [8]:
pos_weight = (y==0).sum() / (y==1).sum()

# Balanced Random Forest

In [ ]:
hyperparameters = {
    'sampling_strategy': ["auto", "all"],
    'replacement': [False, True],
    'bootstrap': [True, False],
}

# model = BalancedRandomForestClassifier(device='gpu', n_estimators=1000, learning_rate=0.05, num_leaves=50, scale_pos_weight=10)
model = BalancedRandomForestClassifier(n_estimators=300, n_jobs=4)

model.fit(
    X_train,
    y_train,
)

print("\n[Validation Result]")
y_pred = model.predict(X_val)
y_prob = model.predict_proba(X_val)[:, 1]

print(f"F1 score: {f1_score(y_val, y_pred)}")
print(f"ROC-AUC score: {roc_auc_score(y_val, y_prob)}")
print(f"Confusion Matrix: \n{confusion_matrix(y_val, y_pred)}")

test_proba = model.predict_proba(X_test)[:, 1]
result_df = pd.concat([test_id, pd.DataFrame(test_proba, columns=["Response"])], axis=1)
result_df.to_csv(f"b_random_forest.csv", index=False)


# XGBoost

In [ ]:
model = XGBClassifier(
    device='cuda',
    n_estimators=5000,
    learning_rate=0.01,
    scale_pos_weight=pos_weight,
    eval_metric=['auc', 'aucpr'],
    verbosity=1,
    objective='binary:logistic',
    sampling_method='gradient_based',
    tree_method='hist',
    early_stopping_rounds=50,
    enable_categorical=True
)

model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
)

print("\n[Validation Result]")
y_pred = model.predict(X_val)
y_prob = model.predict_proba(X_val)[:, 1]

accuracy = accuracy_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
class_report = classification_report(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_prob)
precision = precision_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f'Accuracy: {accuracy:.5f}')
print(f'Precision: {precision:.5f}')
print(f'ROC-AUC: {roc_auc:.5f}')
print(f'F1 Score: {f1:.5f}')
print(f'Confusion Matrix:\n {conf_matrix}')
print(f'Classification Report:\n {class_report} \n')

test_proba = model.predict_proba(X_test)[:, 1]
result_df = pd.concat([test_id, pd.DataFrame(test_proba, columns=["Response"])], axis=1)
result_df.to_csv(f"xgb-est3k-lr01-binary-wght.csv", index=False)


[0]	validation_0-auc:0.84982	validation_0-aucpr:0.33234	validation_1-auc:0.84593	validation_1-aucpr:0.32561
[1]	validation_0-auc:0.84991	validation_0-aucpr:0.33254	validation_1-auc:0.84594	validation_1-aucpr:0.32553
[2]	validation_0-auc:0.85001	validation_0-aucpr:0.33263	validation_1-auc:0.84690	validation_1-aucpr:0.32665
[3]	validation_0-auc:0.85029	validation_0-aucpr:0.33298	validation_1-auc:0.84643	validation_1-aucpr:0.32607
[4]	validation_0-auc:0.85005	validation_0-aucpr:0.33267	validation_1-auc:0.84694	validation_1-aucpr:0.32668
[5]	validation_0-auc:0.85025	validation_0-aucpr:0.33292	validation_1-auc:0.84716	validation_1-aucpr:0.32697
[6]	validation_0-auc:0.85360	validation_0-aucpr:0.34816	validation_1-auc:0.85028	validation_1-aucpr:0.34424
[7]	validation_0-auc:0.85370	validation_0-aucpr:0.34839	validation_1-auc:0.85038	validation_1-aucpr:0.34459
[8]	validation_0-auc:0.85543	validation_0-aucpr:0.35570	validation_1-auc:0.85296	validation_1-aucpr:0.35636
[9]	validation_0-auc:0.85550

XGBoostError: value -1 for Parameter verbosity exceed bound [0,3]
verbosity: Flag to print out detailed breakdown of runtime.

In [ ]:
kaggle.api.competition_submit(
    file_name='./xgb-est3k-lr05-binary-wght.csv',
    message='xgb-binary',
    competition='playground-series-s4e7'
)

100%|██████████| 151M/151M [00:09<00:00, 17.2MB/s]


Successfully submitted to Binary Classification of Insurance Cross Selling 

# LightGBM

In [ ]:
model = LGBMClassifier(
    device='gpu',
    n_estimators=1000,
    learning_rate=0.05,
    scale_pos_weight=pos_weight,
    num_leaves=32,
    colsample_bytree=0.8,
    subsample=0.8,
    max_depth=8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    min_child_samples=20,
    random_state=42
)

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='auc',
    # early_stopping_rounds=100,
    # verbose=100,
    categorical_feature=categorical_features
)

print("\n[Validation Result]")
y_pred = model.predict(X_val)
y_prob = model.predict_proba(X_val)[:, 1]

accuracy = accuracy_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
class_report = classification_report(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_prob)
precision = precision_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f'Accuracy: {accuracy:.5f}')
print(f'Precision: {precision:.5f}')
print(f'ROC-AUC: {roc_auc:.5f}')
print(f'F1 Score: {f1:.5f}')
print(f'Confusion Matrix:\n {conf_matrix}')
print(f'Classification Report:\n {class_report} \n')

test_proba = model.predict_proba(X_test)[:, 1]
result_df_1 = pd.concat([test_id, pd.DataFrame(test_proba, columns=["Response"])], axis=1)
result_df.to_csv(f"lgb-est1k-lr05-wght.csv", index=False)


[LightGBM] [Info] Number of positive: 1411369, number of negative: 10063429
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 742
[LightGBM] [Info] Number of data points in the train set: 11474798, number of used features: 10


LightGBMError: No OpenCL device found

In [ ]:
kaggle.api.competition_submit(
    file_name='./xgb-est3k-lr05-wght.csv',
    message='xgb',
    competition='playground-series-s4e7'
)

100%|██████████| 148M/148M [00:02<00:00, 53.5MB/s]


Successfully submitted to Binary Classification of Insurance Cross Selling 

# Catboost

In [ ]:
model = CatBoostClassifier(
    iterations=5_000,
    scale_pos_weight=pos_weight,
    early_stopping_rounds=50,
    learning_rate=0.05,
    depth=9,
    verbose=500,
    eval_metric="AUC",
    eval_set=[(X_val, y_val)],
    task_type="GPU",
    cat_features=categorical_features,
    # objective='logloss:hints=skip_train~false'
)

model.fit(
    X_train,
    y_train,
)

print("\n[Validation Result]")
y_pred = model.predict(X_val)
y_prob = model.predict_proba(X_val)[:, 1]

accuracy = accuracy_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
class_report = classification_report(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_prob)
precision = precision_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f'Accuracy: {accuracy:.5f}')
print(f'Precision: {precision:.5f}')
print(f'ROC-AUC: {roc_auc:.5f}')
print(f'F1 Score: {f1:.5f}')
print(f'Confusion Matrix:\n {conf_matrix}')
print(f'Classification Report:\n {class_report} \n')

test_proba = model.predict_proba(X_test)[:, 1]
result_df = pd.concat([test_id, pd.DataFrame(test_proba, columns=["Response"])], axis=1)
result_df.to_csv(f"cat-lr05-wght.csv", index=False)


Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 792ms	remaining: 1h 5m 57s
500:	total: 6m 45s	remaining: 1h 40s
1000:	total: 13m 30s	remaining: 53m 57s
1500:	total: 20m 17s	remaining: 47m 18s
2000:	total: 27m 11s	remaining: 40m 45s
2500:	total: 34m 9s	remaining: 34m 7s
3000:	total: 41m 7s	remaining: 27m 23s
3500:	total: 48m 4s	remaining: 20m 35s
4000:	total: 55m	remaining: 13m 44s
4500:	total: 1h 1m 58s	remaining: 6m 52s
4999:	total: 1h 8m 57s	remaining: 0us

[Validation Result]
Accuracy: 0.72613
Precision: 0.29988
ROC-AUC: 0.87649
F1 Score: 0.45219
Confusion Matrix:
 [[18393  7917]
 [  299  3391]]
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.70      0.82     26310
           1       0.30      0.92      0.45      3690

    accuracy                           0.73     30000
   macro avg       0.64      0.81      0.63     30000
weighted avg       0.90      0.73      0.77     30000
 



In [ ]:
X_train[categorical_features].asdtype, X_val[categorical_features].asdtype = np.int64, np.int64

In [ ]:
X_train[categorical_features] = X_train[categorical_features].astype('int32')
X_val[categorical_features] = X_val[categorical_features].astype('int32')
X_test[categorical_features] = X_test[categorical_features].astype('int32')

In [ ]:
model = CatBoostClassifier(
    iterations=1_000,
    scale_pos_weight=pos_weight,
    early_stopping_rounds=100,
    learning_rate=0.05,
    depth=8,
    verbose=100,
    eval_metric="AUC",
    task_type="GPU",
    # bootstrap_type='Bernoulli',
    # bootstrap_type='MVS',
    # bootstrap_type='Bayesian',
    bootstrap_type='Poisson',
    # subsample=0.8,
    # colsample_bylevel=0.8,
    random_seed=42,
    cat_features=categorical_features,
    l2_leaf_reg=3
)

model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val)
)

print("\n[Validation Result]")
y_pred = model.predict(X_val)
y_prob = model.predict_proba(X_val)[:, 1]

accuracy = accuracy_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
class_report = classification_report(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_prob)
precision = precision_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f'Accuracy: {accuracy:.5f}')
print(f'Precision: {precision:.5f}')
print(f'ROC-AUC: {roc_auc:.5f}')
print(f'F1 Score: {f1:.5f}')
print(f'Confusion Matrix:\n {conf_matrix}')
print(f'Classification Report:\n {class_report} \n')

test_proba = model.predict_proba(X_test)[:, 1]
result_df = pd.concat([test_id, pd.DataFrame(test_proba, columns=["Response"])], axis=1)
result_df.to_csv(f"cat-lr05-wght_Poisson.csv", index=False)


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8381048	best: 0.8381048 (0)	total: 710ms	remaining: 11m 49s
100:	test: 0.8651933	best: 0.8651933 (100)	total: 1m 17s	remaining: 11m 27s
200:	test: 0.8692578	best: 0.8692578 (200)	total: 2m 30s	remaining: 9m 59s
300:	test: 0.8714003	best: 0.8714003 (300)	total: 3m 42s	remaining: 8m 36s
400:	test: 0.8724954	best: 0.8724954 (400)	total: 4m 54s	remaining: 7m 19s
500:	test: 0.8732567	best: 0.8732567 (500)	total: 6m 6s	remaining: 6m 5s
600:	test: 0.8738153	best: 0.8738153 (600)	total: 7m 19s	remaining: 4m 51s
700:	test: 0.8742562	best: 0.8742562 (700)	total: 8m 33s	remaining: 3m 38s
800:	test: 0.8745518	best: 0.8745518 (800)	total: 9m 45s	remaining: 2m 25s
900:	test: 0.8747676	best: 0.8747676 (900)	total: 10m 59s	remaining: 1m 12s
999:	test: 0.8749707	best: 0.8749712 (998)	total: 12m 12s	remaining: 0us
bestTest = 0.874971211
bestIteration = 998
Shrink model to first 999 iterations.

[Validation Result]
Accuracy: 0.72053
Precision: 0.29651
ROC-AUC: 0.87497
F1 Score: 0.44929
Confusi

In [ ]:
test_proba = model.predict_proba(X_test)[:, 1]
result_df = pd.concat([test_id, pd.DataFrame(test_proba, columns=["Response"])], axis=1)
result_df.to_csv(f"cat-lr05-wght.csv", index=False)

kaggle.api.competition_submit(
    file_name='./cat-lr05-wght.csv',
    message='catb',
    competition='playground-series-s4e7'
)

100%|██████████| 214M/214M [00:03<00:00, 63.9MB/s]


Successfully submitted to Binary Classification of Insurance Cross Selling 

# AutoML (Optuna)

In [12]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from catboost import CatBoostClassifier, Pool

In [ ]:
def objective_XGB(trial):
    param = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'tree_method': 'gpu_hist',  # Use GPU acceleration if available
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0)
    }

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_val, label=y_val)

    bst = xgb.train(param, dtrain, evals=[(dvalid, 'eval')], early_stopping_rounds=50, verbose=0)

    preds = bst.predict(dvalid)
    roc_auc = roc_auc_score(y_val, preds)

    return roc_auc

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_XGB, n_trials=50)

print("Best hyperparameters: ", study.best_params)
print("Best ROC-AUC: ", study.best_value)

In [ ]:
def objective_CatB(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 1000, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
        'depth': trial.suggest_int('depth', 6, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 1e-3, 10.0, log=True),
        # 'border_count': trial.suggest_int('border_count', 32, 255),
        'border_count': trial.suggest_int('border_count', 100, 300),
        'cat_features': categorical_features,
        'eval_metric': 'AUC',
        'task_type': 'GPU',
        'scale_pos_weight': pos_weight,
    }

    train_pool = Pool(data=X_train,
                      label=y_train,
                      cat_features=categorical_features)
    valid_pool = Pool(data=X_val,
                      label=y_val,
                      cat_features=categorical_features)

    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=50, verbose=0)

    preds = model.predict_proba(valid_pool)[:, 1]
    roc_auc = roc_auc_score(y_val, preds)

    return roc_auc

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_CatB, n_trials=30)

print("Best hyperparameters: ", study.best_params)
print("Best ROC-AUC: ", study.best_value)

[I 2024-07-31 05:45:35,933] A new study created in memory with name: no-name-76e23dd2-5f84-4514-9392-c33ed6d6cfda
Default metric period is 5 because AUC is/are not implemented for GPU
[I 2024-07-31 06:03:09,605] Trial 0 finished with value: 0.8695438842073712 and parameters: {'iterations': 1635, 'learning_rate': 0.05708621750411288, 'depth': 7, 'l2_leaf_reg': 0.6144202219998995, 'bagging_temperature': 0.649272057123948, 'random_strength': 0.5782135855665859, 'border_count': 45}. Best is trial 0 with value: 0.8695438842073712.
Default metric period is 5 because AUC is/are not implemented for GPU
[I 2024-07-31 06:25:32,133] Trial 1 finished with value: 0.8785515827032083 and parameters: {'iterations': 1786, 'learning_rate': 0.05731492680697495, 'depth': 8, 'l2_leaf_reg': 0.4740993788059741, 'bagging_temperature': 0.2501771727360552, 'random_strength': 0.015723027012935237, 'border_count': 202}. Best is trial 1 with value: 0.8785515827032083.
Default metric period is 5 because AUC is/are 

KeyboardInterrupt: 

In [ ]:
best_params = study.best_params
best_params['cat_features'] = categorical_features
best_params['eval_metric'] = 'AUC'
best_params['task_type'] = 'GPU'
best_params['scale_pos_weight'] = pos_weight

final_model = CatBoostClassifier(**best_params)
final_model.fit(X_train, y_train, verbose=0)

# Save the model if needed
final_model.save_model('catboost_model.cbm')

In [23]:
import lightgbm as lgb
from lightgbm import early_stopping

def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'n_estimators': 2000,
        'learning_rate': 0.06,
        'num_leaves': 150,
        'max_depth': trial.suggest_int('max_depth', 10, 15),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 10.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-3, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-3, 10.0, log=True),
        'scale_pos_weight': pos_weight,
        'verbosity': -1,
        'early_stopping_rounds': 50,
        # 'device_type': 'gpu'
    }

    model = LGBMClassifier(**param)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='auc',
        categorical_feature=categorical_features,
        # callbacks=[early_stopping(stopping_rounds=50)]
    )

    preds = model.predict_proba(X_val)[:, 1]
    roc_auc = roc_auc_score(y_val, preds)

    return roc_auc


In [24]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best hyperparameters: ", study.best_params)
print("Best ROC-AUC: ", study.best_value)

[I 2024-07-31 14:13:32,927] A new study created in memory with name: no-name-5002cf88-9757-40dd-b0a3-12f0c0fd4482
[I 2024-07-31 14:16:02,301] Trial 0 finished with value: 0.8819350015811065 and parameters: {'max_depth': 13, 'min_child_weight': 0.3152845416471369, 'subsample': 0.9700017214411523, 'colsample_bytree': 0.905597562428771, 'min_split_gain': 4.243345631148074, 'lambda_l1': 0.21015709105076513, 'lambda_l2': 0.00695558241256704}. Best is trial 0 with value: 0.8819350015811065.
[I 2024-07-31 14:22:37,780] Trial 1 finished with value: 0.8828415473626421 and parameters: {'max_depth': 10, 'min_child_weight': 0.2137622431724438, 'subsample': 0.6136526344368827, 'colsample_bytree': 0.5244824602633918, 'min_split_gain': 0.11649245653249651, 'lambda_l1': 0.8283315966173306, 'lambda_l2': 1.3748197359496048}. Best is trial 1 with value: 0.8828415473626421.
[I 2024-07-31 14:25:13,371] Trial 2 finished with value: 0.8801388489749588 and parameters: {'max_depth': 5, 'min_child_weight': 0.16

Best hyperparameters:  {'max_depth': 10, 'min_child_weight': 0.2137622431724438, 'subsample': 0.6136526344368827, 'colsample_bytree': 0.5244824602633918, 'min_split_gain': 0.11649245653249651, 'lambda_l1': 0.8283315966173306, 'lambda_l2': 1.3748197359496048}
Best ROC-AUC:  0.8828415473626421


In [1]:
data = [
    {'max_depth': 12, 'min_child_weight': 0.01084451186398457, 'subsample': 0.9389741886610761, 'colsample_bytree': 0.6149882207433213, 'min_split_gain': 1.3559419088562674, 'lambda_l1': 0.0020127341082242286, 'lambda_l2': 0.0018355133277479058, 'value': 0.8823315863907403},
    {'max_depth': 1, 'min_child_weight': 2.781145846949944, 'subsample': 0.8742965735619104, 'colsample_bytree': 0.8015042787999667, 'min_split_gain': 3.51674710018088, 'lambda_l1': 0.8335089265043885, 'lambda_l2': 0.019464413668667035, 'value': 0.8213958648138363},
    {'max_depth': 7, 'min_child_weight': 6.278415222835872, 'subsample': 0.6386648977642144, 'colsample_bytree': 0.8385860746509141, 'min_split_gain': 4.024252396948009, 'lambda_l1': 0.0046285824987061025, 'lambda_l2': 0.09358517186030585, 'value': 0.8815790362768698},
    {'max_depth': 12, 'min_child_weight': 3.9182883681569, 'subsample': 0.9497932137597787, 'colsample_bytree': 0.9352844450911151, 'min_split_gain': 1.9817178762415677, 'lambda_l1': 1.8792965697553625, 'lambda_l2': 0.058218664130768386, 'value': 0.8821716319595732},
    {'max_depth': 13, 'min_child_weight': 2.8430086438640934, 'subsample': 0.7792444154854545, 'colsample_bytree': 0.9680133339611907, 'min_split_gain': 9.329466208242431, 'lambda_l1': 0.01952632222413794, 'lambda_l2': 7.980124134650288, 'value': 0.8598019239029333},
    {'max_depth': 13, 'min_child_weight': 0.02488252678100016, 'subsample': 0.9704267834323408, 'colsample_bytree': 0.8018762666145962, 'min_split_gain': 0.7857803223155924, 'lambda_l1': 0.48280278615625166, 'lambda_l2': 0.06473559263613629, 'value': 0.8773595415923753},
    {'max_depth': 10, 'min_child_weight': 0.88647666704409, 'subsample': 0.6073752138760544, 'colsample_bytree': 0.6888701201258522, 'min_split_gain': 0.6349510943977632, 'lambda_l1': 0.045518768980535716, 'lambda_l2': 0.2559753731517955, 'value': 0.866401602119404},
    {'max_depth': 1, 'min_child_weight': 1.976068716208597, 'subsample': 0.9013137723605686, 'colsample_bytree': 0.6711536892267596, 'min_split_gain': 2.1975328586779472, 'lambda_l1': 0.2188705960510443, 'lambda_l2': 0.28084707287490446, 'value': 0.8558013326617493},
    {'max_depth': 2, 'min_child_weight': 0.2995916727286638, 'subsample': 0.63131037816282, 'colsample_bytree': 0.6738821836060003, 'min_split_gain': 6.250200291793658, 'lambda_l1': 1.7695945111280516, 'lambda_l2': 1.1531962228441128, 'value': 0.8746631985324035},
    {'max_depth': 2, 'min_child_weight': 0.3496218949301366, 'subsample': 0.8893364864514273, 'colsample_bytree': 0.9131237719969809, 'min_split_gain': 8.487321206439146, 'lambda_l1': 0.00415462807662946, 'lambda_l2': 0.03454289232850359, 'value': 0.872473381271251},
    {'max_depth': 7, 'min_child_weight': 0.0017605281898034886, 'subsample': 0.7577373610550613, 'colsample_bytree': 0.5116022450923617, 'min_split_gain': 6.40251933120642, 'lambda_l1': 0.0011375013286476108, 'lambda_l2': 0.0014463283035316059, 'value': 0.8808742180732336},
    {'max_depth': 15, 'min_child_weight': 0.019085524684778536, 'subsample': 0.9984637522477071, 'colsample_bytree': 0.5432331983824266, 'min_split_gain': 2.242168595379944, 'lambda_l1': 4.88213004470035, 'lambda_l2': 0.0010002001000998506, 'value': 0.881735411329788},
    {'max_depth': 10, 'min_child_weight': 0.003348586674468389, 'subsample': 0.5180565530693659, 'colsample_bytree': 0.5877999556360125, 'min_split_gain': 0.03470741876450045, 'lambda_l1': 9.148932318634245, 'lambda_l2': 0.0075091298375662766, 'value': 0.8827051292747818},
    {'max_depth': 9, 'min_child_weight': 0.0011798762649489877, 'subsample': 0.6876811594473322, 'colsample_bytree': 0.590059128571498, 'min_split_gain': 0.023153556580429502, 'lambda_l1': 9.116329183429157, 'lambda_l2': 0.007258994158499323, 'value': 0.8825440109019105},
    {'max_depth': 9, 'min_child_weight': 0.0015170467207583137, 'subsample': 0.5219511942128593, 'colsample_bytree': 0.5878033819658992, 'min_split_gain': 3.313057902942765, 'lambda_l1': 8.506794082564065, 'lambda_l2': 0.007944221019168954, 'value': 0.8815896405068193},
    {'max_depth': 5, 'min_child_weight': 0.005067505944852287, 'subsample': 0.5095912056386492, 'colsample_bytree': 0.6052527154256876, 'min_split_gain': 0.06158593285767891, 'lambda_l1': 9.068962579285062, 'lambda_l2': 0.006366789108710993, 'value': 0.8756569729893423},
    {'max_depth': 5, 'min_child_weight': 0.06088907317622535, 'subsample': 0.6924514386188249, 'colsample_bytree': 0.5000339100663973, 'min_split_gain': 5.48814065923349, 'lambda_l1': 2.7662151060382127, 'lambda_l2': 0.005856341108890574, 'value': 0.8803051175323613},
    {'max_depth': 9, 'min_child_weight': 0.004069249143005502, 'subsample': 0.8195710748258425, 'colsample_bytree': 0.7405474325137685, 'min_split_gain': 0.006152944979951374, 'lambda_l1': 0.17245237135405772, 'lambda_l2': 0.013870618921090037, 'value': 0.8813116129450916},
    {'max_depth': 7, 'min_child_weight': 0.004035480021711974, 'subsample': 0.5707895117917465, 'colsample_bytree': 0.5684356023476127, 'min_split_gain': 4.668076630209155, 'lambda_l1': 0.42426602675768665, 'lambda_l2': 0.003093675285445843, 'value': 0.8812654312404014},
    {'max_depth': -1, 'min_child_weight': 0.001062430087073481, 'subsample': 0.6912810405069415, 'colsample_bytree': 0.636886954489486, 'min_split_gain': 2.817439392557059, 'lambda_l1': 1.4093596875111272, 'lambda_l2': 0.2510812577297422, 'value': 0.8799633049352159},
    {'max_depth': 10, 'min_child_weight': 0.07132729534453994, 'subsample': 0.6899513035687695, 'colsample_bytree': 0.7214701754328658, 'min_split_gain': 1.4099759892057344, 'lambda_l1': 0.06182562754038719, 'lambda_l2': 0.003452766803372002, 'value': 0.8649697014644034},
    {'max_depth': 11, 'min_child_weight': 0.011380671713604924, 'subsample': 0.827886797563513, 'colsample_bytree': 0.6238094664286015, 'min_split_gain': 1.2485489067316307, 'lambda_l1': 0.001128290767316947, 'lambda_l2': 0.0021538170332820283, 'value': 0.8823573888152414},
    {'max_depth': 11, 'min_child_weight': 0.008634049319591806, 'subsample': 0.8400766873086131, 'colsample_bytree': 0.5417610636869841, 'min_split_gain': 1.234664817910971, 'lambda_l1': 0.008976835461217077, 'lambda_l2': 0.011812371320626892, 'value': 0.8821974034829668},
    {'max_depth': 15, 'min_child_weight': 0.001984516175357261, 'subsample': 0.7258188928659439, 'colsample_bytree': 0.6338537078123337, 'min_split_gain': 0.11715803994703741, 'lambda_l1': 3.8079291028829583, 'lambda_l2': 0.0030231773954317556, 'value': 0.8828300315500304},
    {'max_depth': 15, 'min_child_weight': 0.0022505877591069874, 'subsample': 0.7146842400844163, 'colsample_bytree': 0.6504717417681166, 'min_split_gain': 0.34596467517033497, 'lambda_l1': 4.057125842057695, 'lambda_l2': 0.024168471115195326, 'value': 0.8826086457177761},
    {'max_depth': 15, 'min_child_weight': 0.002554206279189029, 'subsample': 0.7316496777510064, 'colsample_bytree': 0.7167535249489778, 'min_split_gain': 7.686938558005174, 'lambda_l1': 3.4536126085249403, 'lambda_l2': 0.028800169688026407, 'value': 0.8815154160473571},
    {'max_depth': 14, 'min_child_weight': 0.03326978112707651, 'subsample': 0.5657075111453089, 'colsample_bytree': 0.6468331606510684, 'min_split_gain': 2.68060604020096, 'lambda_l1': 0.8451241476524021, 'lambda_l2': 0.003533374583492052, 'value': 0.8818444510366807},
    {'max_depth': 14, 'min_child_weight': 0.0029614833207088653, 'subsample': 0.7876377372355298, 'colsample_bytree': 0.7630788888793709, 'min_split_gain': 0.6519691009993451, 'lambda_l1': 4.944809697939844, 'lambda_l2': 0.0194811491560771, 'value': 0.8821251978958407},
    {'max_depth': 15, 'min_child_weight': 0.007293672338515415, 'subsample': 0.7214956498303536, 'colsample_bytree': 0.5506972858029655, 'min_split_gain': 1.733543611026581, 'lambda_l1': 4.260350508967373, 'lambda_l2': 0.03470452391164133, 'value': 0.8819848759681059},
    {'max_depth': 13, 'min_child_weight': 0.014374885197795381, 'subsample': 0.6526882216416048, 'colsample_bytree': 0.6439453682594111, 'min_split_gain': 0.8918896037433773, 'lambda_l1': 0.9363735870808234, 'lambda_l2': 0.0015880153965073778, 'value': 0.8825163441106095},
    {'value': 0.8819350015811065, 'max_depth': 13, 'min_child_weight': 0.3152845416471369, 'subsample': 0.9700017214411523, 'colsample_bytree': 0.905597562428771, 'min_split_gain': 4.243345631148074, 'lambda_l1': 0.21015709105076513, 'lambda_l2': 0.00695558241256704},
    {'value': 0.8828415473626421, 'max_depth': 10, 'min_child_weight': 0.2137622431724438, 'subsample': 0.6136526344368827, 'colsample_bytree': 0.5244824602633918, 'min_split_gain': 0.11649245653249651, 'lambda_l1': 0.8283315966173306, 'lambda_l2': 1.3748197359496048},
    {'value': 0.8801388489749588, 'max_depth': 5, 'min_child_weight': 0.16878383083011494, 'subsample': 0.7250516989804938, 'colsample_bytree': 0.6666156105886696, 'min_split_gain': 9.204080939327405, 'lambda_l1': 0.22420991237877852, 'lambda_l2': 0.23534834067034077},
    {'value': 0.8815415171825607, 'max_depth': 9, 'min_child_weight': 0.001115726272637966, 'subsample': 0.9519128185421171, 'colsample_bytree': 0.935825272530749, 'min_split_gain': 4.813673688899003, 'lambda_l1': 0.08593368002414023, 'lambda_l2': 0.21125127787928816},
    {'value': 0.881327187103114, 'max_depth': 9, 'min_child_weight': 0.037915859789753155, 'subsample': 0.7788341421614275, 'colsample_bytree': 0.7342553961526734, 'min_split_gain': 7.560522329138183, 'lambda_l1': 0.6398366771945407, 'lambda_l2': 0.02073254436100961},
    {'value': 0.8804654479269991, 'max_depth': 5, 'min_child_weight': 0.3407836303422483, 'subsample': 0.6255885387451103, 'colsample_bytree': 0.8717431518969407, 'min_split_gain': 7.880540864322261, 'lambda_l1': 0.08253338185698493, 'lambda_l2': 0.020215051197244316},
    {'value': 0.8819174291514864, 'max_depth': 12, 'min_child_weight': 2.087675105592135, 'subsample': 0.6505350691314944, 'colsample_bytree': 0.794126690908063, 'min_split_gain': 3.030386895285211, 'lambda_l1': 5.696785511383941, 'lambda_l2': 1.0477505993751335},
    {'value': 0.8816853103346691, 'max_depth': 14, 'min_child_weight': 0.13064618862800428, 'subsample': 0.9724842247628513, 'colsample_bytree': 0.5139564363532969, 'min_split_gain': 3.29686517272751, 'lambda_l1': 0.011827120076090361, 'lambda_l2': 1.9840764688073187},
    {'value': 0.8809555240364263, 'max_depth': 7, 'min_child_weight': 0.026191672719785706, 'subsample': 0.9055372504949254, 'colsample_bytree': 0.8173987451405775, 'min_split_gain': 7.457872379772653, 'lambda_l1': 1.783651299897775, 'lambda_l2': 0.17978371073715538},
    {'value': 0.8826327485813817, 'max_depth': 11, 'min_child_weight': 0.026611565472664137, 'subsample': 0.53217511646515, 'colsample_bytree': 0.9473543205232133, 'min_split_gain': 0.889304643344947, 'lambda_l1': 0.01654627184695451, 'lambda_l2': 1.4732923165359193},
    {'value': 0.8824899854661794, 'max_depth': 15, 'min_child_weight': 6.8192087086607565, 'subsample': 0.5076893951542822, 'colsample_bytree': 0.5193086246528353, 'min_split_gain': 0.2291582526410293, 'lambda_l1': 0.001460961726985631, 'lambda_l2': 9.100406813706728},
    {'value': 0.8824592388645286, 'max_depth': 11, 'min_child_weight': 0.006074036432032634, 'subsample': 0.5120156024305097, 'colsample_bytree': 0.9921854888947162, 'min_split_gain': 0.03727159708134978, 'lambda_l1': 0.011884726667454054, 'lambda_l2': 1.8517044811755072},
    {'value': 0.8821078829754471, 'max_depth': 10, 'min_child_weight': 0.008813647470510467, 'subsample': 0.5918064094572636, 'colsample_bytree': 0.6130566633684786, 'min_split_gain': 1.708201070625331, 'lambda_l1': 0.0068049529960391925, 'lambda_l2': 8.823340296510246},
    {'value': 0.8820680926497596, 'max_depth': 8, 'min_child_weight': 0.7887476551725919, 'subsample': 0.5613180736953806, 'colsample_bytree': 0.6181807165080412, 'min_split_gain': 1.6069951743249777, 'lambda_l1': 1.3770244518769126, 'lambda_l2': 0.6196429034477633},
    {'value': 0.8824603616047565, 'max_depth': 11, 'min_child_weight': 0.04094922899329776, 'subsample': 0.7275929895610627, 'colsample_bytree': 0.7247707662562184, 'min_split_gain': 1.6436740038025168, 'lambda_l1': 0.031265291763140135, 'lambda_l2': 0.05975227406265452},
    {'value': 0.8812261971346432, 'max_depth': 7, 'min_child_weight': 0.00739820751787647, 'subsample': 0.6633794326771671, 'colsample_bytree': 0.9956895181353513, 'min_split_gain': 5.8210539390268385, 'lambda_l1': 0.0023755415841472767, 'lambda_l2': 2.976116878988941},
    {'value': 0.8824273386215429, 'max_depth': 12, 'min_child_weight': 1.0469902333014827, 'subsample': 0.8046306923371516, 'colsample_bytree': 0.6017424731459887, 'min_split_gain': 0.8221368801078575, 'lambda_l1': 9.744398162307052, 'lambda_l2': 0.0010692169918545555},
    {'value': 0.8820647965316598, 'max_depth': 10, 'min_child_weight': 0.0010748207944587848, 'subsample': 0.5720419987261233, 'colsample_bytree': 0.8290470947867556, 'min_split_gain': 2.6382819075628197, 'lambda_l1': 0.03832945740620586, 'lambda_l2': 0.6419276174955991},
    {'value': 0.8824415634312177, 'max_depth': 13, 'min_child_weight': 0.018559978897181025, 'subsample': 0.6793077640343831, 'colsample_bytree': 0.7032153890324165, 'min_split_gain': 1.0870328530251172, 'lambda_l1': 0.4967855763195806, 'lambda_l2': 4.227565384252279},
    {'value': 0.8818576767105565, 'max_depth': 11, 'min_child_weight': 0.06884776558083232, 'subsample': 0.8574107134555006, 'colsample_bytree': 0.5631346986633422, 'min_split_gain': 2.648458340312358, 'lambda_l1': 2.7715747471336702, 'lambda_l2': 0.06685447068133633},
    {'value': 0.8816213553431619, 'max_depth': 9, 'min_child_weight': 0.003259904656774106, 'subsample': 0.5409562226357707, 'colsample_bytree': 0.6669855429867638, 'min_split_gain': 5.713027175734662, 'lambda_l1': 0.03757614391657282, 'lambda_l2': 0.48271388902934875},
    {'value': 0.8825075475954304, 'max_depth': 15, 'min_child_weight': 9.476704705518404, 'subsample': 0.5170724094798831, 'colsample_bytree': 0.5152137591011963, 'min_split_gain': 0.011733834814177169, 'lambda_l1': 0.0012149912127233835, 'lambda_l2': 7.574987944143365},
    {'value': 0.8827116494083983, 'max_depth': 15, 'min_child_weight': 5.815336916688014, 'subsample': 0.610013038955573, 'colsample_bytree': 0.5568854376220905, 'min_split_gain': 0.1834339443660854, 'lambda_l1': 0.004037011837775488, 'lambda_l2': 4.8732160804339095},
    {'value': 0.8825600897780168, 'max_depth': 14, 'min_child_weight': 2.8062463609932373, 'subsample': 0.6108034969949738, 'colsample_bytree': 0.5562101773149878, 'min_split_gain': 0.9535540150461815, 'lambda_l1': 0.003764920717978906, 'lambda_l2': 1.0595161127488717},
    {'value': 0.8819571061731143, 'max_depth': 13, 'min_child_weight': 0.44638975088618965, 'subsample': 0.6933670179598499, 'colsample_bytree': 0.5769731514178212, 'min_split_gain': 2.0097445195239425, 'lambda_l1': 0.0072062801469303895, 'lambda_l2': 3.5866512113458655},
    {'value': 0.8817695982547054, 'max_depth': 12, 'min_child_weight': 3.0506914438776667, 'subsample': 0.6145166157146491, 'colsample_bytree': 0.7767990245865246, 'min_split_gain': 3.9263888958600557, 'lambda_l1': 0.01449155223350541, 'lambda_l2': 1.4897756225779388},
    {'value': 0.8824298931130703, 'max_depth': 7, 'min_child_weight': 0.07751310264389014, 'subsample': 0.5767390990006529, 'colsample_bytree': 0.6596543907224228, 'min_split_gain': 0.6694538129255997, 'lambda_l1': 0.0036001917428876967, 'lambda_l2': 3.8364860323161416},
    {'value': 0.8822858836532113, 'max_depth': 8, 'min_child_weight': 0.013107872824850137, 'subsample': 0.5429086154932027, 'colsample_bytree': 0.9492681771194005, 'min_split_gain': 1.9480440671660024, 'lambda_l1': 0.020833100752017917, 'lambda_l2': 0.40376497929948457},
    {'value': 0.8821051018758, 'max_depth': 10, 'min_child_weight': 1.293695717385002, 'subsample': 0.6404213154010289, 'colsample_bytree': 0.5434956495450527, 'min_split_gain': 1.0958105063187227, 'lambda_l1': 0.6496014198317813, 'lambda_l2': 5.0936570697096055},
    {'value': 0.8820195985122148, 'max_depth': 14, 'min_child_weight': 0.17362477136960677, 'subsample': 0.6987938397438638, 'colsample_bytree': 0.8603993360144675, 'min_split_gain': 3.7548519153639877, 'lambda_l1': 0.22166440821565528, 'lambda_l2': 0.004272465119088852},
    {'value': 0.8809422005090443, 'max_depth': 6, 'min_child_weight': 4.295513501666297, 'subsample': 0.766869777389888, 'colsample_bytree': 0.6432824502591944, 'min_split_gain': 4.700876014413558, 'lambda_l1': 0.16443970399096675, 'lambda_l2': 0.11991354191998653},
    {'value': 0.8825042205762232, 'max_depth': 14, 'min_child_weight': 4.19437557876113, 'subsample': 0.5963496362811638, 'colsample_bytree': 0.5607007515099823, 'min_split_gain': 0.6948384820755618, 'lambda_l1': 0.005280827704693825, 'lambda_l2': 1.141514904528626},
    {'value': 0.8821074503599464, 'max_depth': 15, 'min_child_weight': 0.6255816883925264, 'subsample': 0.6108310785794682, 'colsample_bytree': 0.5019659823137159, 'min_split_gain': 1.205064488335864, 'lambda_l1': 0.003461800353835957, 'lambda_l2': 0.8236510809734798},
    {'value': 0.8819132369012781, 'max_depth': 14, 'min_child_weight': 1.8087369102442798, 'subsample': 0.5412454889355759, 'colsample_bytree': 0.5900181079758886, 'min_split_gain': 2.4587385022620616, 'lambda_l1': 0.001875987766916051, 'lambda_l2': 0.3354141829461562},
    {'value': 0.8810674117953645, 'max_depth': 13, 'min_child_weight': 0.2671226188418381, 'subsample': 0.6373702176254248, 'colsample_bytree': 0.5488374617262918, 'min_split_gain': 9.84113741644773, 'lambda_l1': 0.00359133236696708, 'lambda_l2': 2.162923961322671},
    {'value': 0.8824956197680563, 'max_depth': 15, 'min_child_weight': 0.0030474120165069974, 'subsample': 0.7219334455122515, 'colsample_bytree': 0.9044160747815708, 'min_split_gain': 0.4762668009166211, 'lambda_l1': 0.06728039536488234, 'lambda_l2': 1.1756120800916872},
    {'value': 0.8825857994991962, 'max_depth': 13, 'min_child_weight': 6.452522410648174, 'subsample': 0.5604205113115939, 'colsample_bytree': 0.68488731455289, 'min_split_gain': 1.23102571129433, 'lambda_l1': 0.0071763701297735665, 'lambda_l2': 2.745547627347059},
    {'value': 0.8826102628757188, 'max_depth': 12, 'min_child_weight': 9.76922408419262, 'subsample': 0.5497263423571741, 'colsample_bytree': 0.6986121291668961, 'min_split_gain': 1.3440537558028596, 'lambda_l1': 0.3567737572893101, 'lambda_l2': 2.5491715814553135},
    {'value': 0.8821558414938008, 'max_depth': 11, 'min_child_weight': 0.22692321948324856, 'subsample': 0.5306858019604477, 'colsample_bytree': 0.7497831505680721, 'min_split_gain': 2.318763919012051, 'lambda_l1': 0.12653216702131279, 'lambda_l2': 5.358226415596503},
    {'value': 0.8820135521955751, 'max_depth': 9, 'min_child_weight': 0.04986734487792169, 'subsample': 0.6625740884205826, 'colsample_bytree': 0.7844048605517734, 'min_split_gain': 3.33567860269041, 'lambda_l1': 0.3859118382097202, 'lambda_l2': 0.2567922718782146},
    {'value': 0.8825725429242129, 'max_depth': 12, 'min_child_weight': 0.11601894548301936, 'subsample': 0.5049553272923007, 'colsample_bytree': 0.6295248306940597, 'min_split_gain': 0.2978737768251276, 'lambda_l1': 0.9834101628365199, 'lambda_l2': 0.02574146681901848},
    {'value': 0.8823655776086458, 'max_depth': 13, 'min_child_weight': 9.598063912984479, 'subsample': 0.5620886397857405, 'colsample_bytree': 0.7192331917853876, 'min_split_gain': 1.4060264069568875, 'lambda_l1': 0.3079413102137391, 'lambda_l2': 2.0696136983594426},
    {'value': 0.8827899219129023, 'max_depth': 12, 'min_child_weight': 5.626070380185088, 'subsample': 0.5799228083628905, 'colsample_bytree': 0.6876880205017113, 'min_split_gain': 0.04067464551656497, 'lambda_l1': 0.00853701993682581, 'lambda_l2': 2.626285514977478},
    {'value': 0.8826934074547892, 'max_depth': 11, 'min_child_weight': 5.48546929697717, 'subsample': 0.5903026046703324, 'colsample_bytree': 0.7598841027842894, 'min_split_gain': 0.09624249887410352, 'lambda_l1': 0.06441257900862545, 'lambda_l2': 5.778075108014945},
    {'value': 0.8823513218978636, 'max_depth': 10, 'min_child_weight': 5.262503534398446, 'subsample': 0.5924172503645493, 'colsample_bytree': 0.9209400217495579, 'min_split_gain': 0.05046162367427742, 'lambda_l1': 0.05262889895529489, 'lambda_l2': 7.141807111003033},
    {'value': 0.8824072323011334, 'max_depth': 11, 'min_child_weight': 1.9746759403659249, 'subsample': 0.628215861589663, 'colsample_bytree': 0.8612717900472738, 'min_split_gain': 0.5719970064931709, 'lambda_l1': 0.017178076232915913, 'lambda_l2': 1.5426067490493387},
    {'value': 0.8826293494595913, 'max_depth': 9, 'min_child_weight': 0.024090469406410162, 'subsample': 0.5851969811346303, 'colsample_bytree': 0.5282968310265499, 'min_split_gain': 0.004583820187058607, 'lambda_l1': 0.025606610026863978, 'lambda_l2': 4.998709218339894},
    {'value': 0.8815346365360271, 'max_depth': 11, 'min_child_weight': 2.9422389218135496, 'subsample': 0.8124143249369219, 'colsample_bytree': 0.8155494537055298, 'min_split_gain': 7.116620808780929, 'lambda_l1': 0.010382970004623892, 'lambda_l2': 9.09229395303455},
    {'value': 0.8810420677372872, 'max_depth': 8, 'min_child_weight': 1.3344602020234058, 'subsample': 0.9437574704557279, 'colsample_bytree': 0.9564446158379973, 'min_split_gain': 8.631564859875212, 'lambda_l1': 0.06480589854402972, 'lambda_l2': 0.7157906361214026},
    {'value': 0.8825623249581033, 'max_depth': 10, 'min_child_weight': 3.9626748911461376, 'subsample': 0.6603770457575142, 'colsample_bytree': 0.8912018790438719, 'min_split_gain': 0.48267688337424264, 'lambda_l1': 3.063239570949989, 'lambda_l2': 6.370548059641822},
    {'value': 0.8819350015811065, 'max_depth': 13, 'min_child_weight': 0.3152845416471369, 'subsample': 0.9700017214411523, 'colsample_bytree': 0.905597562428771, 'min_split_gain': 4.243345631148074, 'lambda_l1': 0.21015709105076513, 'lambda_l2': 0.00695558241256704},
    {'value': 0.8828415473626421, 'max_depth': 10, 'min_child_weight': 0.2137622431724438, 'subsample': 0.6136526344368827, 'colsample_bytree': 0.5244824602633918, 'min_split_gain': 0.11649245653249651, 'lambda_l1': 0.8283315966173306, 'lambda_l2': 1.3748197359496048},
    {'value': 0.8801388489749588, 'max_depth': 5, 'min_child_weight': 0.16878383083011494, 'subsample': 0.7250516989804938, 'colsample_bytree': 0.6666156105886696, 'min_split_gain': 9.204080939327405, 'lambda_l1': 0.22420991237877852, 'lambda_l2': 0.23534834067034077},
    {'value': 0.8815415171825607, 'max_depth': 9, 'min_child_weight': 0.001115726272637966, 'subsample': 0.9519128185421171, 'colsample_bytree': 0.935825272530749, 'min_split_gain': 4.813673688899003, 'lambda_l1': 0.08593368002414023, 'lambda_l2': 0.21125127787928816},
    {'value': 0.881327187103114, 'max_depth': 9, 'min_child_weight': 0.037915859789753155, 'subsample': 0.7788341421614275, 'colsample_bytree': 0.7342553961526734, 'min_split_gain': 7.560522329138183, 'lambda_l1': 0.6398366771945407, 'lambda_l2': 0.02073254436100961},
    {'value': 0.8804654479269991, 'max_depth': 5, 'min_child_weight': 0.3407836303422483, 'subsample': 0.6255885387451103, 'colsample_bytree': 0.8717431518969407, 'min_split_gain': 7.880540864322261, 'lambda_l1': 0.08253338185698493, 'lambda_l2': 0.020215051197244316},
    {'value': 0.8819174291514864, 'max_depth': 12, 'min_child_weight': 2.087675105592135, 'subsample': 0.6505350691314944, 'colsample_bytree': 0.794126690908063, 'min_split_gain': 3.030386895285211, 'lambda_l1': 5.696785511383941, 'lambda_l2': 1.0477505993751335},
    {'value': 0.8816853103346691, 'max_depth': 14, 'min_child_weight': 0.13064618862800428, 'subsample': 0.9724842247628513, 'colsample_bytree': 0.5139564363532969, 'min_split_gain': 3.29686517272751, 'lambda_l1': 0.011827120076090361, 'lambda_l2': 1.9840764688073187},
    {'value': 0.8809555240364263, 'max_depth': 7, 'min_child_weight': 0.026191672719785706, 'subsample': 0.9055372504949254, 'colsample_bytree': 0.8173987451405775, 'min_split_gain': 7.457872379772653, 'lambda_l1': 1.783651299897775, 'lambda_l2': 0.17978371073715538},
    {'value': 0.8826327485813817, 'max_depth': 11, 'min_child_weight': 0.026611565472664137, 'subsample': 0.53217511646515, 'colsample_bytree': 0.9473543205232133, 'min_split_gain': 0.889304643344947, 'lambda_l1': 0.01654627184695451, 'lambda_l2': 1.4732923165359193},
    {'value': 0.8824899854661794, 'max_depth': 15, 'min_child_weight': 6.8192087086607565, 'subsample': 0.5076893951542822, 'colsample_bytree': 0.5193086246528353, 'min_split_gain': 0.2291582526410293, 'lambda_l1': 0.001460961726985631, 'lambda_l2': 9.100406813706728},
    {'value': 0.8824592388645286, 'max_depth': 11, 'min_child_weight': 0.006074036432032634, 'subsample': 0.5120156024305097, 'colsample_bytree': 0.9921854888947162, 'min_split_gain': 0.03727159708134978, 'lambda_l1': 0.011884726667454054, 'lambda_l2': 1.8517044811755072},
    {'value': 0.8821078829754471, 'max_depth': 10, 'min_child_weight': 0.008813647470510467, 'subsample': 0.5918064094572636, 'colsample_bytree': 0.6130566633684786, 'min_split_gain': 1.708201070625331, 'lambda_l1': 0.0068049529960391925, 'lambda_l2': 8.823340296510246},
    {'value': 0.8820680926497596, 'max_depth': 8, 'min_child_weight': 0.7887476551725919, 'subsample': 0.5613180736953806, 'colsample_bytree': 0.6181807165080412, 'min_split_gain': 1.6069951743249777, 'lambda_l1': 1.3770244518769126, 'lambda_l2': 0.6196429034477633},
    {'value': 0.8824603616047565, 'max_depth': 11, 'min_child_weight': 0.04094922899329776, 'subsample': 0.7275929895610627, 'colsample_bytree': 0.7247707662562184, 'min_split_gain': 1.6436740038025168, 'lambda_l1': 0.031265291763140135, 'lambda_l2': 0.05975227406265452},
    {'value': 0.8812261971346432, 'max_depth': 7, 'min_child_weight': 0.00739820751787647, 'subsample': 0.6633794326771671, 'colsample_bytree': 0.9956895181353513, 'min_split_gain': 5.8210539390268385, 'lambda_l1': 0.0023755415841472767, 'lambda_l2': 2.976116878988941},
    {'value': 0.8824273386215429, 'max_depth': 12, 'min_child_weight': 1.0469902333014827, 'subsample': 0.8046306923371516, 'colsample_bytree': 0.6017424731459887, 'min_split_gain': 0.8221368801078575, 'lambda_l1': 9.744398162307052, 'lambda_l2': 0.0010692169918545555},
    {'value': 0.8820647965316598, 'max_depth': 10, 'min_child_weight': 0.0010748207944587848, 'subsample': 0.5720419987261233, 'colsample_bytree': 0.8290470947867556, 'min_split_gain': 2.6382819075628197, 'lambda_l1': 0.03832945740620586, 'lambda_l2': 0.6419276174955991},
    {'value': 0.8824415634312177, 'max_depth': 13, 'min_child_weight': 0.018559978897181025, 'subsample': 0.6793077640343831, 'colsample_bytree': 0.7032153890324165, 'min_split_gain': 1.0870328530251172, 'lambda_l1': 0.4967855763195806, 'lambda_l2': 4.227565384252279},
    {'value': 0.8818576767105565, 'max_depth': 11, 'min_child_weight': 0.06884776558083232, 'subsample': 0.8574107134555006, 'colsample_bytree': 0.5631346986633422, 'min_split_gain': 2.648458340312358, 'lambda_l1': 2.7715747471336702, 'lambda_l2': 0.06685447068133633},
    {'value': 0.8816213553431619, 'max_depth': 9, 'min_child_weight': 0.003259904656774106, 'subsample': 0.5409562226357707, 'colsample_bytree': 0.6669855429867638, 'min_split_gain': 5.713027175734662, 'lambda_l1': 0.03757614391657282, 'lambda_l2': 0.48271388902934875},
    {'value': 0.8825075475954304, 'max_depth': 15, 'min_child_weight': 9.476704705518404, 'subsample': 0.5170724094798831, 'colsample_bytree': 0.5152137591011963, 'min_split_gain': 0.011733834814177169, 'lambda_l1': 0.0012149912127233835, 'lambda_l2': 7.574987944143365},
    {'value': 0.8827116494083983, 'max_depth': 15, 'min_child_weight': 5.815336916688014, 'subsample': 0.610013038955573, 'colsample_bytree': 0.5568854376220905, 'min_split_gain': 0.1834339443660854, 'lambda_l1': 0.004037011837775488, 'lambda_l2': 4.8732160804339095},
    {'value': 0.8825600897780168, 'max_depth': 14, 'min_child_weight': 2.8062463609932373, 'subsample': 0.6108034969949738, 'colsample_bytree': 0.5562101773149878, 'min_split_gain': 0.9535540150461815, 'lambda_l1': 0.003764920717978906, 'lambda_l2': 1.0595161127488717},
    {'value': 0.8819571061731143, 'max_depth': 13, 'min_child_weight': 0.44638975088618965, 'subsample': 0.6933670179598499, 'colsample_bytree': 0.5769731514178212, 'min_split_gain': 2.0097445195239425, 'lambda_l1': 0.0072062801469303895, 'lambda_l2': 3.5866512113458655},
    {'value': 0.8817695982547054, 'max_depth': 12, 'min_child_weight': 3.0506914438776667, 'subsample': 0.6145166157146491, 'colsample_bytree': 0.7767990245865246, 'min_split_gain': 3.9263888958600557, 'lambda_l1': 0.01449155223350541, 'lambda_l2': 1.4897756225779388},
    {'value': 0.8824298931130703, 'max_depth': 7, 'min_child_weight': 0.07751310264389014, 'subsample': 0.5767390990006529, 'colsample_bytree': 0.6596543907224228, 'min_split_gain': 0.6694538129255997, 'lambda_l1': 0.0036001917428876967, 'lambda_l2': 3.8364860323161416},
    {'value': 0.8822858836532113, 'max_depth': 8, 'min_child_weight': 0.013107872824850137, 'subsample': 0.5429086154932027, 'colsample_bytree': 0.9492681771194005, 'min_split_gain': 1.9480440671660024, 'lambda_l1': 0.020833100752017917, 'lambda_l2': 0.40376497929948457},
    {'value': 0.8821051018758, 'max_depth': 10, 'min_child_weight': 1.293695717385002, 'subsample': 0.6404213154010289, 'colsample_bytree': 0.5434956495450527, 'min_split_gain': 1.0958105063187227, 'lambda_l1': 0.6496014198317813, 'lambda_l2': 5.0936570697096055},
    {'value': 0.8820195985122148, 'max_depth': 14, 'min_child_weight': 0.17362477136960677, 'subsample': 0.6987938397438638, 'colsample_bytree': 0.8603993360144675, 'min_split_gain': 3.7548519153639877, 'lambda_l1': 0.22166440821565528, 'lambda_l2': 0.004272465119088852},
    {'value': 0.8809422005090443, 'max_depth': 6, 'min_child_weight': 4.295513501666297, 'subsample': 0.766869777389888, 'colsample_bytree': 0.6432824502591944, 'min_split_gain': 4.700876014413558, 'lambda_l1': 0.16443970399096675, 'lambda_l2': 0.11991354191998653},
    {'value': 0.8825042205762232, 'max_depth': 14, 'min_child_weight': 4.19437557876113, 'subsample': 0.5963496362811638, 'colsample_bytree': 0.5607007515099823, 'min_split_gain': 0.6948384820755618, 'lambda_l1': 0.005280827704693825, 'lambda_l2': 1.141514904528626},
    {'value': 0.8821074503599464, 'max_depth': 15, 'min_child_weight': 0.6255816883925264, 'subsample': 0.6108310785794682, 'colsample_bytree': 0.5019659823137159, 'min_split_gain': 1.205064488335864, 'lambda_l1': 0.003461800353835957, 'lambda_l2': 0.8236510809734798},
    {'value': 0.8819132369012781, 'max_depth': 14, 'min_child_weight': 1.8087369102442798, 'subsample': 0.5412454889355759, 'colsample_bytree': 0.5900181079758886, 'min_split_gain': 2.4587385022620616, 'lambda_l1': 0.001875987766916051, 'lambda_l2': 0.3354141829461562},
    {'value': 0.8810674117953645, 'max_depth': 13, 'min_child_weight': 0.2671226188418381, 'subsample': 0.6373702176254248, 'colsample_bytree': 0.5488374617262918, 'min_split_gain': 9.84113741644773, 'lambda_l1': 0.00359133236696708, 'lambda_l2': 2.162923961322671},
    {'value': 0.8824956197680563, 'max_depth': 15, 'min_child_weight': 0.0030474120165069974, 'subsample': 0.7219334455122515, 'colsample_bytree': 0.9044160747815708, 'min_split_gain': 0.4762668009166211, 'lambda_l1': 0.06728039536488234, 'lambda_l2': 1.1756120800916872},
    {'value': 0.8825857994991962, 'max_depth': 13, 'min_child_weight': 6.452522410648174, 'subsample': 0.5604205113115939, 'colsample_bytree': 0.68488731455289, 'min_split_gain': 1.23102571129433, 'lambda_l1': 0.0071763701297735665, 'lambda_l2': 2.745547627347059},
    {'value': 0.8826102628757188, 'max_depth': 12, 'min_child_weight': 9.76922408419262, 'subsample': 0.5497263423571741, 'colsample_bytree': 0.6986121291668961, 'min_split_gain': 1.3440537558028596, 'lambda_l1': 0.3567737572893101, 'lambda_l2': 2.5491715814553135},
    {'value': 0.8821558414938008, 'max_depth': 11, 'min_child_weight': 0.22692321948324856, 'subsample': 0.5306858019604477, 'colsample_bytree': 0.7497831505680721, 'min_split_gain': 2.318763919012051, 'lambda_l1': 0.12653216702131279, 'lambda_l2': 5.358226415596503},
    {'value': 0.8820135521955751, 'max_depth': 9, 'min_child_weight': 0.04986734487792169, 'subsample': 0.6625740884205826, 'colsample_bytree': 0.7844048605517734, 'min_split_gain': 3.33567860269041, 'lambda_l1': 0.3859118382097202, 'lambda_l2': 0.2567922718782146},
    {'value': 0.8825725429242129, 'max_depth': 12, 'min_child_weight': 0.11601894548301936, 'subsample': 0.5049553272923007, 'colsample_bytree': 0.6295248306940597, 'min_split_gain': 0.2978737768251276, 'lambda_l1': 0.9834101628365199, 'lambda_l2': 0.02574146681901848},
    {'value': 0.8823655776086458, 'max_depth': 13, 'min_child_weight': 9.598063912984479, 'subsample': 0.5620886397857405, 'colsample_bytree': 0.7192331917853876, 'min_split_gain': 1.4060264069568875, 'lambda_l1': 0.3079413102137391, 'lambda_l2': 2.0696136983594426},
    {'value': 0.8827899219129023, 'max_depth': 12, 'min_child_weight': 5.626070380185088, 'subsample': 0.5799228083628905, 'colsample_bytree': 0.6876880205017113, 'min_split_gain': 0.04067464551656497, 'lambda_l1': 0.00853701993682581, 'lambda_l2': 2.626285514977478},
    {'value': 0.8826934074547892, 'max_depth': 11, 'min_child_weight': 5.48546929697717, 'subsample': 0.5903026046703324, 'colsample_bytree': 0.7598841027842894, 'min_split_gain': 0.09624249887410352, 'lambda_l1': 0.06441257900862545, 'lambda_l2': 5.778075108014945},
    {'value': 0.8823513218978636, 'max_depth': 10, 'min_child_weight': 5.262503534398446, 'subsample': 0.5924172503645493, 'colsample_bytree': 0.9209400217495579, 'min_split_gain': 0.05046162367427742, 'lambda_l1': 0.05262889895529489, 'lambda_l2': 7.141807111003033},
    {'value': 0.8824072323011334, 'max_depth': 11, 'min_child_weight': 1.9746759403659249, 'subsample': 0.628215861589663, 'colsample_bytree': 0.8612717900472738, 'min_split_gain': 0.5719970064931709, 'lambda_l1': 0.017178076232915913, 'lambda_l2': 1.5426067490493387},
    {'value': 0.8826293494595913, 'max_depth': 9, 'min_child_weight': 0.024090469406410162, 'subsample': 0.5851969811346303, 'colsample_bytree': 0.5282968310265499, 'min_split_gain': 0.004583820187058607, 'lambda_l1': 0.025606610026863978, 'lambda_l2': 4.998709218339894},
    {'value': 0.8815346365360271, 'max_depth': 11, 'min_child_weight': 2.9422389218135496, 'subsample': 0.8124143249369219, 'colsample_bytree': 0.8155494537055298, 'min_split_gain': 7.116620808780929, 'lambda_l1': 0.010382970004623892, 'lambda_l2': 9.09229395303455},
    {'value': 0.8810420677372872, 'max_depth': 8, 'min_child_weight': 1.3344602020234058, 'subsample': 0.9437574704557279, 'colsample_bytree': 0.9564446158379973, 'min_split_gain': 8.631564859875212, 'lambda_l1': 0.06480589854402972, 'lambda_l2': 0.7157906361214026},
    {'value': 0.8825623249581033, 'max_depth': 10, 'min_child_weight': 3.9626748911461376, 'subsample': 0.6603770457575142, 'colsample_bytree': 0.8912018790438719, 'min_split_gain': 0.48267688337424264, 'lambda_l1': 3.063239570949989, 'lambda_l2': 6.370548059641822},
]

In [2]:
import pandas as pd
pd.DataFrame(data).corr()

,max_depth,min_child_weight,subsample,colsample_bytree,min_split_gain,lambda_l1,lambda_l2,value
max_depth,1.000000,0.248587,-0.112459,-0.212574,-0.391482,-0.007179,0.211425,0.398270
min_child_weight,0.248587,1.000000,-0.289395,-0.068285,-0.249346,-0.169463,0.448587,0.020418
subsample,-0.112459,-0.289395,1.000000,0.169883,0.387365,0.068633,-0.300516,-0.223476
colsample_bytree,-0.212574,-0.068285,0.169883,1.000000,0.245621,-0.177687,-0.042590,-0.105072
min_split_gain,-0.391482,-0.249346,0.387365,0.245621,1.000000,-0.139963,-0.189044,-0.160688
lambda_l1,-0.007179,-0.169463,0.068633,-0.177687,-0.139963,1.000000,-0.250648,0.030132
lambda_l2,0.211425,0.448587,-0.300516,-0.042590,-0.189044,-0.250648,1.000000,0.087232
value,0.398270,0.020418,-0.223476,-0.105072,-0.160688,0.030132,0.087232,1.000000


In [3]:
data = [
    {'value': 0.8802054408609461, 'iterations': 1800, 'learning_rate': 0.07817062766722944, 'depth': 9, 'l2_leaf_reg': 1.462575048356281, 'bagging_temperature': 0.38361727764570597, 'random_strength': 0.0021311401022627097, 'border_count': 219},
    {'value': 0.8776226233185936, 'iterations': 1494, 'learning_rate': 0.029016427536828437, 'depth': 7, 'l2_leaf_reg': 0.0011090529669500913, 'bagging_temperature': 0.4424026380710122, 'random_strength': 0.028543746010986987, 'border_count': 233},
    {'value': 0.8802341067880463, 'iterations': 1123, 'learning_rate': 0.05248534611581913, 'depth': 10, 'l2_leaf_reg': 0.003981152818678197, 'bagging_temperature': 0.674895331452293, 'random_strength': 0.88927655808582, 'border_count': 272},
    {'value': 0.8763194772768709, 'iterations': 1882, 'learning_rate': 0.014691793975856108, 'depth': 10, 'l2_leaf_reg': 0.005495611898544276, 'bagging_temperature': 0.4407460960174926, 'random_strength': 0.0433650365118862, 'border_count': 162},
    {'value': 0.8776149495436422, 'iterations': 1312, 'learning_rate': 0.05909640212368699, 'depth': 9, 'l2_leaf_reg': 0.002535185986124485, 'bagging_temperature': 0.6106800782005538, 'random_strength': 0.41152083901630876, 'border_count': 161},
    {'value': 0.8778971384544709, 'iterations': 1197, 'learning_rate': 0.08257266142962064, 'depth': 9, 'l2_leaf_reg': 0.015241508383557168, 'bagging_temperature': 0.7344953765113759, 'random_strength': 0.024470164450380576, 'border_count': 166},
    {'value': 0.8753094591379209, 'iterations': 1355, 'learning_rate': 0.021034852030978116, 'depth': 9, 'l2_leaf_reg': 0.0028818499678861785, 'bagging_temperature': 0.8260990344124016, 'random_strength': 0.790175013196209, 'border_count': 138},
    {'value': 0.8733918188288686, 'iterations': 1765, 'learning_rate': 0.03811813029524214, 'depth': 6, 'l2_leaf_reg': 9.482182932198652, 'bagging_temperature': 0.9648245092493716, 'random_strength': 0.015560572393408263, 'border_count': 105},
    {'value': 0.8735406952131095, 'iterations': 1066, 'learning_rate': 0.016564107221631954, 'depth': 8, 'l2_leaf_reg': 0.4663045223496878, 'bagging_temperature': 0.4435345055964962, 'random_strength': 0.0012817841229436017, 'border_count': 200},
    {'value': 0.8782850658039079, 'iterations': 1949, 'learning_rate': 0.020804330063701436, 'depth': 7, 'l2_leaf_reg': 0.025744391715298465, 'bagging_temperature': 0.46202538932845716, 'random_strength': 0.0030248373480734844, 'border_count': 293},
    {'value': 0.8710087769444779, 'iterations': 1023, 'learning_rate': 0.010237965857458297, 'depth': 10, 'l2_leaf_reg': 0.11817133295789775, 'bagging_temperature': 0.05812077565550944, 'random_strength': 6.71651748320475, 'border_count': 294},
    {'value': 0.8811145462841934, 'iterations': 1670, 'learning_rate': 0.0979865167635557, 'depth': 10, 'l2_leaf_reg': 1.9092804681957374, 'bagging_temperature': 0.25573633889134806, 'random_strength': 0.6296853576087397, 'border_count': 248},
    {'value': 0.8809466708692172, 'iterations': 1626, 'learning_rate': 0.0522751209033116, 'depth': 10, 'l2_leaf_reg': 5.055912678081245, 'bagging_temperature': 0.11891639925228342, 'random_strength': 1.4335776339159239, 'border_count': 259},
    {'value': 0.8813685997369285, 'iterations': 1621, 'learning_rate': 0.09242395464662255, 'depth': 10, 'l2_leaf_reg': 9.905401870805399, 'bagging_temperature': 0.1407659358224826, 'random_strength': 6.237881074597164, 'border_count': 253},
    {'value': 0.8808524482432206, 'iterations': 1610, 'learning_rate': 0.09795981040049875, 'depth': 8, 'l2_leaf_reg': 1.8709167611736106, 'bagging_temperature': 0.2405744098814464, 'random_strength': 9.73211207144895, 'border_count': 249},
    {'value': 0.8790564655931622, 'iterations': 1603, 'learning_rate': 0.06778762705809231, 'depth': 10, 'l2_leaf_reg': 0.48045007164828313, 'bagging_temperature': 0.2502575420260746, 'random_strength': 2.616893154482683, 'border_count': 210},
    {'value': 0.8808700103724716, 'iterations': 1443, 'learning_rate': 0.09898235820115232, 'depth': 9, 'l2_leaf_reg': 3.5352471976634634, 'bagging_temperature': 0.20865043886750081, 'random_strength': 0.1922662458207781, 'border_count': 247},
    {'value': 0.88038157717191, 'iterations': 1719, 'learning_rate': 0.039694826644190384, 'depth': 8, 'l2_leaf_reg': 0.8328452343578161, 'bagging_temperature': 0.3121337227071545, 'random_strength': 3.047862060754498, 'border_count': 281},
    {'value': 0.8773713818666122, 'iterations': 1656, 'learning_rate': 0.041033850501918, 'depth': 7, 'l2_leaf_reg': 0.09647007324855962, 'bagging_temperature': 0.00651400764448995, 'random_strength': 0.15026387219171203, 'border_count': 183},
    {'value': 0.8805723915087877, 'iterations': 1994, 'learning_rate': 0.06949170947016306, 'depth': 10, 'l2_leaf_reg': 0.1591699323829465, 'bagging_temperature': 0.14027480524804192, 'random_strength': 0.3468573935636453, 'border_count': 232},
    {'value': 0.8791205132879911, 'iterations': 1538, 'learning_rate': 0.027591006270031818, 'depth': 8, 'l2_leaf_reg': 8.716130001943935, 'bagging_temperature': 0.1449890947573889, 'random_strength': 4.331800453102943, 'border_count': 267},
    {'value': 0.8815287241241854, 'iterations': 1692, 'learning_rate': 0.05911434781843123, 'depth': 10, 'l2_leaf_reg': 4.284437246753088, 'bagging_temperature': 0.08673982751364728, 'random_strength': 1.3163446939806982, 'border_count': 256},
    {'value': 0.8798564849578562, 'iterations': 1817, 'learning_rate': 0.08197283965665017, 'depth': 10, 'l2_leaf_reg': 3.622213699689094, 'bagging_temperature': 0.3242008352423701, 'random_strength': 1.6515200368841874, 'border_count': 237},
    {'value': 0.8803358332329048, 'iterations': 1703, 'learning_rate': 0.09782030078808356, 'depth': 9, 'l2_leaf_reg': 1.6227731025089618, 'bagging_temperature': 0.004425546783295625, 'random_strength': 0.5232377675198875, 'border_count': 220},
    {'value': 0.8811795725140832, 'iterations': 1527, 'learning_rate': 0.06175291231717164, 'depth': 10, 'l2_leaf_reg': 4.360444772128833, 'bagging_temperature': 0.10320110486919613, 'random_strength': 0.08527987442823044, 'border_count': 259},
    {'value': 0.8809835925421208, 'iterations': 1502, 'learning_rate': 0.04885053871799596, 'depth': 9, 'l2_leaf_reg': 6.071514599115097, 'bagging_temperature': 0.10180922326029361, 'random_strength': 0.08269721225022533, 'border_count': 274},
    {'value': 0.8810814099969203, 'iterations': 1369, 'learning_rate': 0.06269395581498102, 'depth': 10, 'l2_leaf_reg': 0.29865384073747725, 'bagging_temperature': 0.17955716904047003, 'random_strength': 0.00901394146937412, 'border_count': 297},
    {'value': 0.8793947606142727, 'iterations': 1442, 'learning_rate': 0.04791730997130659, 'depth': 6, 'l2_leaf_reg': 3.186532471267209, 'bagging_temperature': 0.5600922979700989, 'random_strength': 0.06521121569603308, 'border_count': 256},
    {'value': 0.8785293236056649, 'iterations': 1558, 'learning_rate': 0.07815139441750728, 'depth': 9, 'l2_leaf_reg': 9.98086123357767, 'bagging_temperature': 0.0884481544990407, 'random_strength': 0.20907135345371056, 'border_count': 186},
    {'value': 0.8792609176186782, 'iterations': 1834, 'learning_rate': 0.0348545535933179, 'depth': 9, 'l2_leaf_reg': 0.9187582233015996, 'bagging_temperature': 0.33541627985484807, 'random_strength': 4.898717381901866, 'border_count': 220},

]

In [4]:
pd.DataFrame(data).corr()

,value,iterations,learning_rate,depth,l2_leaf_reg,bagging_temperature,random_strength,border_count
value,1.000000,0.383021,0.660221,0.320074,0.168892,-0.419499,0.002800,0.475705
iterations,0.383021,1.000000,0.162997,-0.060912,0.155248,-0.192532,-0.040373,-0.044548
learning_rate,0.660221,0.162997,1.000000,0.325552,0.218293,-0.275088,0.078549,0.070011
depth,0.320074,-0.060912,0.325552,1.000000,-0.080041,-0.381030,0.117339,0.226078
l2_leaf_reg,0.168892,0.155248,0.218293,-0.080041,1.000000,-0.180529,0.130315,-0.026934
bagging_temperature,-0.419499,-0.192532,-0.275088,-0.381030,-0.180529,1.000000,-0.264823,-0.555677
random_strength,0.002800,-0.040373,0.078549,0.117339,0.130315,-0.264823,1.000000,0.297975
border_count,0.475705,-0.044548,0.070011,0.226078,-0.026934,-0.555677,0.297975,1.000000
